## Install Necessary Libraries ##

In [ ]:
# Upgrade pip
%pip install --upgrade pip

# Install OpenCV for image processing
%pip install opencv-python

# Install PyTorch and related packages
# Adjust the versions according to your CUDA and Python versions

# For CUDA 11.7 and Python 3.10:

# Install torch==2.0.0, torchvision==0.15.1, torchaudio==2.0.0
%pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.0+cu117 --index-url https://download.pytorch.org/whl/cu117

# Install gitpython if not already installed
%pip install gitpython

# Install dependencies
%pip install cython
%pip install git+https://github.com/facebookresearch/fvcore
%pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

# Clone the Detectron2 repository and install it
import os
if not os.path.exists('detectron2'):
    !git clone https://github.com/facebookresearch/detectron2.git

# Install Detectron2
%cd detectron2
%pip install -e .
%cd ..

# Install TensorFlow and Keras for deep learning models
%pip install tensorflow==2.10.0

# Install pickleshare to fix the IPython warning
%pip install pickleshare

# Install additional required libraries
%pip install numpy matplotlib pandas scikit-learn tqdm Pillow

# Install geospatial libraries
%pip install geopandas rasterio shapely

# Install any other utility libraries if needed
%pip install jsonschema pyyaml

# Install IPython kernel to ensure compatibility
%pip install ipykernel

# After installations, you may need to restart the kernel
print("All libraries installed. Please restart the kernel to ensure all packages are loaded.")


## Import Necessary Libraries ##

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import random

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog


## Define functions to load and register the dataset for Detectron2 ##

In [ ]:
def get_roof_dicts(img_dir):
    # Placeholder function to load your dataset
    # You need to implement this function to return a list of dictionaries
    # Each dictionary corresponds to an image and contains:
    # - file_name: path to the image file
    # - height: image height
    # - width: image width
    # - annotations: list of annotations, each with:
    #   - segmentation: polygon of the object
    #   - bbox: bounding box [x, y, width, height]
    #   - bbox_mode: BoxMode.XYWH_ABS
    #   - category_id: class id (0 for rooftops)

    dataset_dicts = []
    # Implement data loading here
    return dataset_dicts

# Register the dataset
for d in ["train", "val"]:
    DatasetCatalog.register("roof_" + d, lambda d=d: get_roof_dicts("path/to/your/dataset/" + d))
    MetadataCatalog.get("roof_" + d).set(thing_classes=["roof"])

roof_metadata = MetadataCatalog.get("roof_train")


## Visualize sample data to ensure it's loaded correctly ##

In [ ]:
dataset_dicts = get_roof_dicts("path/to/your/dataset/train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=roof_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(12, 8))
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.axis('off')
    plt.show()


## Configure and train the Mask R-CNN model ##

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("roof_train",)
cfg.DATASETS.TEST = ("roof_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Learning rate
cfg.SOLVER.MAX_ITER = 1000    # Adjust number of iterations as needed
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only rooftops

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


## Perform inference using the trained model ##

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # Set the testing threshold
predictor = DefaultPredictor(cfg)

test_image = cv2.imread("path/to/your/test/image.jpg")
outputs = predictor(test_image)


## Visualize the inference results ##

In [ ]:
v = Visualizer(test_image[:, :, ::-1], metadata=roof_metadata, scale=0.8)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(12, 8))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis('off')
plt.show()


## Prepare data generators for roof type classification ##

In [ ]:
# Define directories
train_dir = 'path/to/roof_type/train'
val_dir = 'path/to/roof_type/val'

# Image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    vertical_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)


## Build and train the roof type classification model ##

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)


## Perform obstacle detection using edge detection and color thresholding ##

In [ ]:
# Load the roof image (segmented roof area)
roof_image = cv2.imread('path/to/segmented/roof_image.jpg')

# Convert to grayscale
gray = cv2.cvtColor(roof_image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Find contours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on a mask
obstacle_mask = np.zeros_like(gray)
cv2.drawContours(obstacle_mask, contours, -1, 255, thickness=cv2.FILLED)

# Optional: Use color thresholding to refine obstacle detection
hsv = cv2.cvtColor(roof_image, cv2.COLOR_BGR2HSV)
lower_val = np.array([0, 0, 0])
upper_val = np.array([180, 255, 30])  # Adjust values based on obstacle colors
color_mask = cv2.inRange(hsv, lower_val, upper_val)

# Combine edge and color masks
combined_mask = cv2.bitwise_or(obstacle_mask, color_mask)

# Visualize the obstacle mask
plt.figure(figsize=(12, 8))
plt.imshow(combined_mask, cmap='gray')
plt.axis('off')
plt.title('Obstacle Mask')
plt.show()


## Calculate the effective roof area excluding obstacles ##

In [ ]:
# Assume roof_mask is a binary mask of the roof area
roof_mask = cv2.imread('path/to/roof_mask.jpg', cv2.IMREAD_GRAYSCALE)
roof_area_pixels = cv2.countNonZero(roof_mask)

# Obstacle area in pixels
obstacle_area_pixels = cv2.countNonZero(combined_mask)

# Effective area in pixels
effective_area_pixels = roof_area_pixels - obstacle_area_pixels

# Spatial resolution in meters per pixel (you need to define this based on your data)
resolution = 0.1  # Example: each pixel represents 0.1 meters

# Convert to square meters
effective_area_m2 = effective_area_pixels * (resolution ** 2)

print(f"Effective roof area available for solar panels: {effective_area_m2:.2f} square meters")


## Prepare data generators for roof material classification ##

In [ ]:
# Define directories
train_dir = 'path/to/roof_material/train'
val_dir = 'path/to/roof_material/val'

# Image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    vertical_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

num_material_classes = len(train_generator.class_indices)


## Build and train the roof material classification model ##

In [ ]:
base_model_material = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = base_model_material.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_material_classes, activation='softmax')(x)
material_model = Model(inputs=base_model_material.input, outputs=predictions)

# Freeze base model layers
for layer in base_model_material.layers:
    layer.trainable = False

# Compile the model
material_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
material_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)


## Detect roof faces and calculate orientation angles ##

In [ ]:
# Load the roof image
roof_image = cv2.imread('path/to/roof_image.jpg')
gray = cv2.cvtColor(roof_image, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Use Hough Line Transform to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

# Create an empty image to draw lines
line_image = np.zeros_like(roof_image)

# Draw lines on the image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Overlay lines on the original image
overlay_image = cv2.addWeighted(roof_image, 0.8, line_image, 1, 0)

# Display the image with detected lines
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(overlay_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Detected Roof Faces')
plt.show()

# Placeholder for normal vector calculation
# In reality, you would need 3D coordinates of the roof faces
# For demonstration, let's assume a normal vector
normal_vector = np.array([0.5, 0.5, 0.7071])  # Example normal vector

# Function to calculate orientation
import math

def calculate_orientation(normal_vector):
    nx, ny, nz = normal_vector
    azimuth = math.degrees(math.atan2(ny, nx)) % 360
    tilt = math.degrees(math.acos(nz / np.linalg.norm(normal_vector)))
    return azimuth, tilt

azimuth_angle, tilt_angle = calculate_orientation(normal_vector)

print(f"Azimuth Angle: {azimuth_angle:.2f} degrees")
print(f"Tilt Angle: {tilt_angle:.2f} degrees")


## Determine if the roof face is suitable for solar panel installation ##

In [ ]:
if 30 <= tilt_angle <= 40:
    print("The roof face has an ideal tilt angle for solar panels.")
    suitable_area_m2 = effective_area_m2  # Assuming the entire effective area is suitable
else:
    print("The roof face does not have an ideal tilt angle for solar panels.")
    suitable_area_m2 = 0

print(f"Suitable area for solar panels: {suitable_area_m2:.2f} square meters")


## Estimate potential energy production ##

In [ ]:
# Assuming average solar irradiance and panel efficiency
solar_irradiance = 1000  # W/m^2 (average peak sun hours)
panel_efficiency = 0.17  # 17% efficiency
system_losses = 0.14     # 14% losses

# Calculate the estimated energy production
daily_energy_kWh = suitable_area_m2 * solar_irradiance * panel_efficiency * (1 - system_losses) * 4 / 1000

print(f"Estimated daily energy production: {daily_energy_kWh:.2f} kWh")
